In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import math
from sklearn.cluster import KMeans
%matplotlib inline
import sys

In [2]:
sys.path.append("..")
from framework import perspective_transform
from framework import visualize
from framework import edges_detection

In [3]:
# data_path = "/home/avk/study/1C/Diplom/data/NewData/samples/"
data_path = "/home/avk/study/1C/Diplom/PerspectiveDistortion/SampleData/"

In [4]:
# # for file_num, filename in enumerate(os.listdir(data_path)):
# #     full_file_path = os.path.join(data_path, filename)
# #     if file_num == 5:
# # #     if file_num == 2:
# #         break
# # file_num
# full_file_path = "/home/avk/study/1C/Diplom/PerspectiveDistortion/SampleData/IMG_20190317_170456.jpg"

In [5]:
for file_num, filename in enumerate(os.listdir(data_path)):
    full_file_path = os.path.join(data_path, filename)
#     if file_num == 3: #4 # 6 # 7 # 5 -unreal!
# to show: 
# 0 - example before-after
# 1 - easy case
# 2 - interesting additional edges
# 3 - horizontal
# 4 vs 5 - laminat
# 6 - white background
# 7 - folder
# 8 - angle
# 9 - additional blur
# 10 - fail
    if file_num == 3:
        break
file_num

3

In [6]:
full_file_path = "/home/avk/study/1C/Diplom/data/source2/other_12.png"

In [7]:
initial_image = cv2.imread(full_file_path)
visualize.visualize_image(initial_image)
initial_image.shape

(2448, 3264, 3)

# Уменьшение изображения

In [8]:
resized_image, resizing_ratio = edges_detection.reduce_image_size(initial_image)
visualize.visualize_image(resized_image)
resized_image.shape

(300, 400, 3)

# Размытие текста

In [9]:
# for times in range(1, 8):
#     smoothed_image = edges_detection.blur_image(resized_image, cv2.medianBlur, 5, times)
#     visualize.visualize_image(smoothed_image, str(times))

In [10]:
smoothed_image = edges_detection.blur_image(resized_image, cv2.medianBlur, 5, 3)
visualize.visualize_image(smoothed_image)

# Выделение границ

In [11]:
# edges = cv2.Canny(smoothed_image, 10, 50)
# visualize.visualize_image(edges)

In [12]:
edges = edges_detection.canny_edge_detector(smoothed_image)
visualize.visualize_image(edges)

For CannyEdgeDetection greyscale image was chosen


# Построение линий Хафа

In [13]:
hough_lines = edges_detection.find_hough_lines(edges)
print(len(hough_lines[0]), len(hough_lines[1]))
assert(min(len(hough_lines[0]), len(hough_lines[1])) >= 2)

3 2


In [14]:
cdst = edges_detection.draw_lines_on_edges(edges, hough_lines)
visualize.visualize_image(cdst)
print(hough_lines)

[[array([[248.      ,   1.553343]], dtype=float32), array([[-2.       ,  1.6231562]], dtype=float32), array([[250.       ,   1.5358897]], dtype=float32)], [array([[3.9000000e+01, 3.4906585e-02]], dtype=float32), array([[-371.      ,    3.106686]], dtype=float32)]]


# Поиск пересечений линий

In [15]:
intersections = edges_detection.segmented_intersections(hough_lines)
len(intersections)

6

In [16]:
for point in intersections:
    cv2.circle(cdst, point, 3, (0, 255, 0), -1)
visualize.visualize_image(cdst)

# Кластеризация точек пересечения

In [17]:
corners = KMeans(n_clusters=4).fit(np.array(intersections)).cluster_centers_

In [18]:
for corner in corners:
    corner = tuple(np.array(corner, dtype=int))
    cv2.circle(cdst, corner, 3, (255, 0, 0), -1)
visualize.visualize_image(cdst)

# Работа с изначальным (неуменьшенным) изображением

In [19]:
corners = np.array(corners * resizing_ratio, dtype=int)
corners

array([[ 318,    0],
       [3096, 1950],
       [ 244, 2027],
       [3035,  138]])

In [20]:
restored = perspective_transform.remove_perspective_distortion(initial_image, corners)
visualize.visualize_image(restored)
restored.shape, restored.shape[0] / restored.shape[1]

((1963, 2940, 3), 0.667687074829932)